# First Simple GAN

Tutorial from: https://www.youtube.com/watch?v=OljTVUVzPpM&list=PLhhyoLH6IjfwIp8bZnzX8QR30TRcHO8Va&index=2

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter

In [2]:
class Discriminator(nn.Module):

    def __init__(self, img_dim,out_features=128,act_val=0.1):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            nn.Linear(img_dim, out_features),
            nn.LeakyReLU(act_val),
            nn.Linear(out_features,1), # linear to 1 dim, 0 or 1, real or fake, equivalently
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.disc(x)

In [3]:
class Generator(nn.Module):

    def __init__(self, z_dim, img_dim, out_features = 256, act_val=0.1):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            nn.Linear(z_dim, out_features),
            nn.LeakyReLU(act_val),
            nn.Linear(out_features, img_dim),
            nn.Tanh() # Output of pixel values are between -1 and 1, input will be normalized to be between (-1,1)
        )
    def forward(self, x):
        return self.gen(x)

In [4]:
# Hyper Parameters

# GANs are highly sensitive to HyperParameters

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
lr = 3e-4 # Best LR for Adam according to Andrej Karpathy
z_dim = 128
img_dim = 28 * 28 * 1 # mnist
batch_size = 32
num_epochs = 50

In [5]:
disc = Discriminator(img_dim).to(device)
gen = Generator(z_dim, img_dim).to(device)
fixed_noise = torch.randn((batch_size, z_dim)).to(device)
mean_mnist = 0.1307
deviation_mnist = 0.3081
transforms = transforms.Compose(
    [ transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))]
)

In [6]:
dataset = datasets.MNIST(root="dataset/", transform=transforms, download=True)
loader = DataLoader(dataset, batch_size= batch_size, shuffle=True)


In [7]:
opt_disc = optim.Adam (disc.parameters(), lr=lr) # Discriminator Optimizer
opt_gen = optim.Adam (gen.parameters(), lr=lr) # Generator Optimizer
criterion = nn.BCELoss()

In [8]:
writer_fake = SummaryWriter(f"runs/GAN_MNIST/fake")
writer_real = SummaryWriter(f"runs/GAN_MNIST/real")
step = 0

In [9]:
for epoch in range(num_epochs):
    for batch_idx, (real, _) in enumerate(loader):
        real = real.view(-1, 784).to(device)
        assert batch_size == real.shape[0], f"Batch Size not being correctly structured"

        # Train Discriminator: max log(D(real)) + log(1 - D(G(z)))

        noise = torch.randn(batch_size, z_dim).to(device)
        fake = gen(noise)

        disc_real = disc(real).view(-1)
        lossD_real = criterion(disc_real, torch.ones_like(disc_real))
        disc_fake = disc(fake.detach()).view(-1)
        lossD_fake = criterion(disc_fake, torch.zeros_like(disc_fake))

        lossD = (lossD_real + lossD_fake) /2
        disc.zero_grad()
        lossD.backward()
        opt_disc.step()

        # Train Generator: min log(1 - D(G(z))) <-> max (log(D(G(z))))

        output = disc(fake).view(-1)
        lossG = criterion(output, torch.ones_like(output))

        gen.zero_grad()
        lossG.backward()
        opt_gen.step()

        if batch_idx == 0:
            print(
                f"Epoch [{epoch}/{num_epochs}] Batch {batch_idx}/{len(loader)} \
                      Loss D: {lossD:.4f}, loss G: {lossG:.4f}"
            )

            with torch.no_grad():
                fake = gen(fixed_noise).reshape(-1, 1, 28, 28)
                data = real.reshape(-1, 1, 28, 28)
                img_grid_fake = torchvision.utils.make_grid(fake, normalize=True)
                img_grid_real = torchvision.utils.make_grid(data, normalize=True)

                writer_fake.add_image(
                    "Mnist Fake Images", img_grid_fake, global_step=step
                )
                writer_real.add_image(
                    "Mnist Real Images", img_grid_real, global_step=step
                )
                step += 1




Epoch [0/50] Batch 0/1875                       Loss D: 0.7112, loss G: 0.6681
Epoch [1/50] Batch 0/1875                       Loss D: 0.4323, loss G: 1.3318
Epoch [2/50] Batch 0/1875                       Loss D: 0.8477, loss G: 0.6369
Epoch [3/50] Batch 0/1875                       Loss D: 0.4386, loss G: 1.2163
Epoch [4/50] Batch 0/1875                       Loss D: 0.7589, loss G: 0.8492
Epoch [5/50] Batch 0/1875                       Loss D: 1.1714, loss G: 0.3923
Epoch [6/50] Batch 0/1875                       Loss D: 0.6705, loss G: 0.7390
Epoch [7/50] Batch 0/1875                       Loss D: 0.6777, loss G: 1.0467
Epoch [8/50] Batch 0/1875                       Loss D: 0.7800, loss G: 0.7734
Epoch [9/50] Batch 0/1875                       Loss D: 0.8070, loss G: 1.0438
Epoch [10/50] Batch 0/1875                       Loss D: 0.5375, loss G: 1.1117
Epoch [11/50] Batch 0/1875                       Loss D: 0.4899, loss G: 1.2870
Epoch [12/50] Batch 0/1875                       L

KeyboardInterrupt: 